<h1 style="color:#1B4F72;">UANL OCR</h1>
<b>Pedro Antonio Martinez Gonzalez 1798942</b>
<br><br>
<i>El objetivo es procesar 1,110 archivos PDF todos escaneados, extraer la matricula y nombre del alumno para posteriormente modificar el nombre del archivo PDF por la matricula y nombre.</i>

<h1 style="color:#1B4F72;">Instalar Dependencias</h1>
<h3>Ejecutar los siguientes comandos</h3>
<i>pip install pdf2image</i><br>
<i>pip install opencv-python</i><br>
<i>pip install pytesseract</i><br>

<h3>Instalar pyteseract en windows</h3>
<i>Descargar el ejecutable de </i><a>https://github.com/UB-Mannheim/tesseract/wiki</a><i>, es recomendable instalarlo para todos los usuarios del sistema y en la ruta predeterminada de instalación, de lo contrario será necesario cambiar la línea 10 del main.py por la nueva ruta de tesseract.exe.</i>

In [1]:
from pdf2image import convert_from_path
import cv2
import pytesseract
from pytesseract import Output
import matplotlib.pyplot as plt
from os import scandir
import re
import os

pytesseract.pytesseract.tesseract_cmd = r'C:\Program Files\Tesseract-OCR\tesseract.exe'
custom_config = r'--oem 3 --psm 6'

<h3>Esta función devuelve una lista con los archivos contenidos en un directorio</h3>

In [2]:
def ls2(path): 
    return [obj.name for obj in scandir(path) if obj.is_file()]

<h3>Convierte un archivo PDF a una Imagen</h3>

In [3]:
def pdf_to_image(pdf_name):
    images = convert_from_path(pdf_path = 'PDF/' + pdf_name , poppler_path = r'poppler-21.02.0\Library\bin')
    images[0].save(pdf_name + '.jpg', 'JPEG')

<img src=https://pythonforundergradengineers.com/posts/pdf/images/pdf_to_dir.png width=504 height=274>

<h3>Esta función preparar la imagen para ser procesada por el OCR</h3>

In [4]:
def image_pre_process(jpg_name):
    image = cv2.imread(jpg_name + '.jpg')
    os.remove(jpg_name + '.jpg')

    gray_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    threshold_img = cv2.threshold(gray_image, 0, 255, cv2.THRESH_BINARY | cv2.THRESH_OTSU)[1]
    
    details = pytesseract.image_to_data(threshold_img, output_type=Output.DICT, config=custom_config, lang='eng')
    total_boxes = len(details['text'])
    
    for sequence_number in range(total_boxes):
        if details['text'][sequence_number] == "Numero" and details['text'][sequence_number + 1] == "de" and details['text'][sequence_number + 2] == "Alumno":
            if int(details['conf'][sequence_number]) >30:
                (x, y, w, h) = (details['left'][sequence_number], details['top'][sequence_number], details['width'][sequence_number], details['height'][sequence_number])
                threshold_img = threshold_img[y - 20 : y + h + 80 , x - 30 : x + w + 1300]

    return threshold_img

<img src="https://scontent-qro1-1.xx.fbcdn.net/v/t1.15752-9/151792787_482624906240857_5248179418294355423_n.png?_nc_cat=100&ccb=3&_nc_sid=ae9488&_nc_eui2=AeHuhavq_Ycipxx0wyWSGmwpLNfc5VkI0rss19zlWQjSu-0nnb9-85Ngj14avJtJ44NhxTfjCpf3ES1bHES9iQhP&_nc_ohc=x6sualnmvJoAX8x_Qmk&_nc_ht=scontent-qro1-1.xx&oh=caa3c6a0c5b9d239cef8407be3bc818e&oe=60615457">

<h3>Procesa la imagen y extrae los valores de matricula y nombre</h3>

In [5]:
def image_process(threshold_img):
    details = pytesseract.image_to_data(threshold_img, output_type=Output.DICT, config=custom_config, lang='eng')

    name = ''
    matricula = ''
    
    for i in range(len(details['text'])):
        if re.findall("\d{7}", details['text'][i]) and matricula == '':
            matricula = details['text'][i]
            continue

        if bool(re.findall("\d{7}", details['text'][i])) == False and matricula != '':
            name = name + details['text'][i]

        if re.findall("\d{7}", details['text'][i]) and matricula != '':
            break
    
    name = name.replace(",", " ")
    return matricula + " " +  name

<h3>Esta función cambia el nombre del archivo</h3>

In [6]:
def change_name(old_file_name, new_file_name):
    os.rename('PDF\{}'.format(old_file_name), 
              'PDF\{}.pdf'.format(new_file_name))

<h3>Aquí se ejecutan todas las funciones definidas anteriormente</h3>

In [8]:
files = ls2('PDF/')

for file in files:
    pdf_to_image(file)
    
    threshold_img = image_pre_process(file)
    new_file_name = image_process(threshold_img)

    change_name(file, new_file_name)
    print("Archivo de Entrada:\t" + file)
    print("Archivo de Salida:\t" + new_file_name + ".pdf\n")

Archivo de Entrada:	20201215-045000001.pdf
Archivo de Salida:	1968610 HERRERA SANCHEZ VICTORIA.pdf



<h1 style="color:#1B4F72;"> Ejecucion</h1>
<i>Para la ejecución es necesario cumplir con las dependencias de la sección “Instalar Dependencias”, lo siguiente es colocar los archivos que desea renombrar en el directorio “PDF”, por último, ejecutar el main.py con el comando “python main.py”</i>
<img src="https://scontent-qro1-1.xx.fbcdn.net/v/t1.15752-9/155914620_1043209652838958_216827113365319928_n.png?_nc_cat=109&ccb=3&_nc_sid=ae9488&_nc_eui2=AeHSaZFZqTvy6SIF5HVJB7zuVfXPyRnOtnlV9c_JGc62ebO4MN5YL_optt8fB3HMFXt4Mmb-qErUpp_GFtVMT7Wb&_nc_ohc=YddR4pbSCYYAX8-SrA5&_nc_ht=scontent-qro1-1.xx&oh=c34adff9c8343a0e0155c32432007596&oe=6062FDD0">